#Calculate the rank of clarify question

In [1]:
from google.colab import drive
drive.mount('/content/drive')
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mounted at /content/drive
Thu Jul 29 20:06:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                              

## Install Package

In [2]:
!pip3 install pandas 
!pip3 install transformers 
!pip3 install tensorflow 
!pip3 install numpy 
!pip3 install seaborn 

     |████████████████████████████████| 2.6 MB 9.2 MB/s 
     |████████████████████████████████| 895 kB 37.9 MB/s 
     |████████████████████████████████| 3.3 MB 40.6 MB/s 
     |████████████████████████████████| 636 kB 46.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


##Import Package

In [3]:
import pandas as pd
import tensorflow as tf
import transformers
from tensorflow.keras import layers
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os

##Set Global Parameter

In [4]:
os.chdir("/content/drive/MyDrive/Colab_File")
print(os.getcwd())
path_test = './NLP/data/test_set.csv'
path_cp = './NLP/result/my_checkpoint.ckpt'

max_length = 35
batch_size = 64

/content/drive/MyDrive/Colab_File


##Network Structure

In [5]:
input_ids = layers.Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
attention_masks = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="attention_masks")
token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids")
bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
bert_model.trainable = False
bert_output = bert_model(
    [input_ids, attention_masks, token_type_ids]
)
sequence_output = bert_output.last_hidden_state
bi_lstm = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(sequence_output)
avg_pool = layers.GlobalAveragePooling1D()(bi_lstm)
max_pool = layers.GlobalMaxPooling1D()(bi_lstm)
concat = layers.concatenate([avg_pool, max_pool])
#layer1 = layers.Dense(256, activation="tanh")(pooler_output)
dropout = layers.Dropout(0.3)(concat)
output = layers.Dense(1, activation="sigmoid")(dropout)
#output = layers.Dense(1, activation="relu")(dropout)

model = tf.keras.models.Model(
    inputs=[input_ids, attention_masks, token_type_ids],
    outputs=output
)

model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

##Data Generator

In [6]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):

    def __init__(self, sentence_pairs, labels, batch_size, max_length, shuffle=True, include_targets=True):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.max_length = max_length
        self.include_targets = include_targets
        self.tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self): # overload len function
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx): # overload getitem function, in oder to retrieve each element
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=self.max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_tensors="tf"
        )

        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            #labels = np.array(self.labels[indexes], dtype="float32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):

        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


#Compute Semantic Score from Bert_LSTM

###load data

In [7]:
model.load_weights(path_cp)
test_df = pd.read_csv(path_test)
print(test_df.head(5))
test_data = BertSemanticDataGenerator(
    sentence_pairs=test_df[['test_query','question']].values.astype("str"),
    labels=None,
    batch_size=6,
    max_length=35,
    shuffle=False,
    include_targets=False
)








                test_query  ...                                           question
0  Tell me about Computers  ...  a total cholesterol of 180 to 200 mgdl 10 to 1...
1  Tell me about Computers  ...  about how many years experience do you want th...
2  Tell me about Computers  ...  according to anima the bible or what other source
3  Tell me about Computers  ...  ae you looking for examples of septic system d...
4  Tell me about Computers  ...                  any specific company on your mind

[5 rows x 3 columns]


##Compute Similarity

In [8]:
test_sim = model.predict(test_data, verbose=1)
print(test_sim)

30659/30659 [==============================] - 1214s 39ms/step
[[6.4482056e-02]
 [1.6689273e-02]
 [1.6252628e-04]
 ...
 [4.1324478e-05]
 [5.7327590e-05]
 [9.8723537e-01]]


##Ranking

In [9]:
path_whole = './NLP/result/whole_result.csv'
path_answer = './NLP/result/answer.txt'
path_unorder = './NLP/result/whole_result_unorder.csv'
m,_=np.shape(test_sim)
test_sim = test_sim.reshape(-1,1)
print("the total ranked sample is:", m)
df_rank = pd.DataFrame(test_sim[:,0], columns=['semantic_score'])
df_score = pd.concat((test_df,df_rank), axis=1)
test_question = df_score['test_query'].unique()
test_num = test_question.shape[0]
df_final_rank = pd.DataFrame(columns=['test_query','question_id', 'question', 'semantic_score'])
answer=pd.DataFrame(columns=['test_query','top_50_ids'])
for i in range(test_num):
  to_store = {}
  this_question = test_question[i]
  rank_candidate = df_score[df_score['test_query'] == this_question]
  rank_candidate = rank_candidate.sort_values(by=['semantic_score'], ascending=False)
  rank_50 = rank_candidate.iloc[:50, 1].tolist()
  rank_out = ','.join(rank_50)
  if i == 0:
    print(len(rank_50))

  to_store['top_50_ids'] = [rank_out]
  to_store['test_query'] = this_question
  df_store=pd.DataFrame(to_store)
  df_final_rank = df_final_rank.append(rank_candidate)
  answer = answer.append(df_store)
df_final_rank.to_csv(path_whole)
answer.to_csv(path_answer, sep='\t', header=False, index=False)
df_score.to_csv(path_unorder, index=False)
df_final_rank.head(51)

the total ranked sample is: 183954
50


,test_query,question_id,question,semantic_score
3940,Tell me about Computers,Q03942,do you want to know what parts are in a computer,0.999731
1565,Tell me about Computers,Q01567,do you like using computers,0.999636
646,Tell me about Computers,Q00648,are you looking for a musical keyboard or a co...,0.999627
1056,Tell me about Computers,Q01058,are you looking for the computer part,0.999252
2083,Tell me about Computers,Q02085,do you want to know how to do computer program...,0.999067
2168,Tell me about Computers,Q02170,do you want to know technical information abou...,0.998550
259,Tell me about Computers,Q00261,are you interested in knowing more about a com...,0.998343
495,Tell me about Computers,Q00497,are you interested in the small single board c...,0.998269
142,Tell me about Computers,Q00144,are you interested in atari 2600,0.998241
783,Tell me about Computers,Q00785,are you looking for computer programming schoo...,0.997601


In [10]:
print(len(df_final_rank[df_final_rank['test_query']=="Tell me about Computers"]))

3999
